# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
partOneData = "../output_data/cities.csv"
vacationDf = pd.read_csv(partOneData)
vacationDf2 = vacationDf.drop(columns=["Unnamed: 0"])
vacationDf2.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,79.00,71,24,19.19,PF,1644198597
1,san patricio,13.3034,-4.8956,71.64,12,0,8.39,ML,1644198882
2,nemuro,43.3236,145.5750,28.83,71,25,11.81,JP,1644198882
3,mahibadhoo,3.7833,72.9667,80.98,65,65,10.54,MV,1644198882
4,fortuna,40.5982,-124.1573,54.19,91,0,0.00,US,1644198883


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key = g_key)
cities = vacationDf2[["Lat", "Lng"]]
humidity = vacationDf2["Humidity"].astype(float)

In [13]:
fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity = 150, point_radius = 1)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

idealCities = vacationDf2.loc[(vacationDf2["Max Temp"] > 70) & (vacationDf2["Max Temp"] < 80) & (vacationDf2["Cloudiness"] == 0) & (vacationDf2["Wind Speed"] < 10)]
idealCities = idealCities.dropna(how = "any")
idealCities.reset_index(inplace = True)

idealCities.head(25)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,san patricio,13.3034,-4.8956,71.64,12,0,8.39,ML,1644198882
1,99,morwell,-38.2333,146.4000,78.35,33,0,6.58,AU,1644198911
2,107,pochutla,15.7432,-96.4661,71.33,83,0,2.53,MX,1644198914
3,117,ode,7.7899,5.7117,70.92,22,0,3.42,NG,1644198917
4,173,san cristobal,13.3034,-4.8956,71.64,12,0,8.39,ML,1644198934
5,332,launceston,-41.4500,147.1667,75.61,44,0,3.00,AU,1644198964
6,384,saint-pierre,-21.3393,55.4781,78.48,78,0,6.91,RE,1644199003
7,410,springbok,-29.6643,17.8865,71.04,46,0,4.36,ZA,1644199011
8,527,oussouye,12.4850,-16.5469,70.95,71,0,5.17,SN,1644199047
9,541,mulchen,-37.7167,-72.2333,71.19,37,0,4.72,CL,1644199052


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotels = []

for i in range(len(idealCities)):
    lat = idealCities.loc[i]["Lat"]
    lng = idealCities.loc[i]["Lat"]
    targetSearch = "hotel"
    targetRadius = 5000
    targetType = "lodging"
    
    params = {"location": f"{lat}, {lng}",
              "radius": targetRadius,
              "type": targetType,
              "keywords": targetSearch,
              "key": g_key
    }
    queryUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lodging = requests.get(queryUrl, params = params).json()
    
    try:
        hotels.append(lodging["results"][0]["name"])
    except:
         hotels.append("np.nan")

idealCities["Hotel Name"] = hotels
idealCities = idealCities.dropna(how = "any")
idealCities.head(10)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,1,san patricio,13.3034,-4.8956,71.64,12,0,8.39,ML,1644198882,np.nan
1,99,morwell,-38.2333,146.4000,78.35,33,0,6.58,AU,1644198911,np.nan
2,107,pochutla,15.7432,-96.4661,71.33,83,0,2.53,MX,1644198914,np.nan
3,117,ode,7.7899,5.7117,70.92,22,0,3.42,NG,1644198917,np.nan
4,173,san cristobal,13.3034,-4.8956,71.64,12,0,8.39,ML,1644198934,np.nan
5,332,launceston,-41.4500,147.1667,75.61,44,0,3.00,AU,1644198964,np.nan
6,384,saint-pierre,-21.3393,55.4781,78.48,78,0,6.91,RE,1644199003,np.nan
7,410,springbok,-29.6643,17.8865,71.04,46,0,4.36,ZA,1644199011,np.nan
8,527,oussouye,12.4850,-16.5469,70.95,71,0,5.17,SN,1644199047,np.nan
9,541,mulchen,-37.7167,-72.2333,71.19,37,0,4.72,CL,1644199052,np.nan


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
